In [3]:
from math import pi
import numpy as np
import gzip
from scipy.stats import multivariate_normal

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


# Problem 1: 
# find $\sigma$ such that $E(\|x\|)=d$, where $x\sim \mathcal{N}(\vec 0, \sigma^2 I_n)$

The theoretical solution is given by the expression:

$$ E(\|x\|) = \sigma \sqrt{2} \frac{\Gamma((n+1)/2)}{\Gamma(n/2)}$$

See for example: see https://stats.stackexchange.com/questions/144893/what-is-the-expected-norm-mathbb-e-lvert-x-rvert-for-a-multivariate-normal

Therefore, for $E(\|x\|)=d$ and $n=3$ we have the following calculation of $\sigma$:

$$\sigma = \frac{\Gamma(3/2)}{\sqrt{2}}d = \frac{\sqrt{\pi}}{2\sqrt{2}}d$$

In [4]:
def get_sigma_square(d):
    return d ** 2 * pi / 8

def get_spherical_gaussian(d):
    mu = [0., 0., 0.]
    sigma_square = get_sigma_square(d)
    cov = [[sigma_square, 0., 0.], [0., sigma_square, 0.], [0., 0., sigma_square]]
    return multivariate_normal(mu, cov)

In [5]:
# Sanity check

pae = 5  # input distance
k = 100000  # number of randomizations
eps = 1e-2  # tolerance

dist = get_spherical_gaussian(pae)
distances = [np.sqrt(np.dot(x,x)) for x in dist.rvs(size=k)]
assert(abs(np.mean(distances) - pae) < eps)

# Problem 2:
# compute the probability mass of $\mathcal{N}(\vec 0, \sigma^2 I_n)$ restricted to the sphere 
# $$S=\{y\in\mathbb{R}^3\;|\; \|y-c\|^2 < r^2\}$$

In [6]:
pae = 7  # input PAE
c = 4  # distance between residues
r = 10  # contact threshold in Angstroms 
k = 100000  # number of randomizations

center = np.array([c, 0., 0.])
dist = get_spherical_gaussian(pae)

contact_prob = np.mean([int(np.dot(y-center, y-center) < r**2) for y in dist.rvs(size=k)])
contact_prob

0.74166

# Wrap up:

# How to compute the contact probability with these scripts?

Given residues C and R, with C being the center of the contact volume:

1) d = PAE(R, C) -- the one that evaluates the expected error on R if we align at C, provided by AlphaFold

2) r = contact volume radius about C -- typically 10Å in our setting

3) c = estimate of the distance between R and C -- provided by AlphaFold

The contact probability, according to the spherical Gaussian model, can be computed with the following function:

In [7]:
def contact_probability(d, r, c, k=10000):

    center = np.array([c, 0., 0.])
    dist = get_spherical_gaussian(d)
    return np.mean([int(np.dot(y-center, y-center) < r**2) for y in dist.rvs(size=k)])

In [8]:
d = 10
r = 10
c = 8

contact_probability(d, r, c)

0.3392

# Test

In [9]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from Bio.PDB.PDBParser import PDBParser
from scipy.stats import pearsonr

In [10]:
# Functions to compute the probability of contact

def cap(r, h):
    """
    Volume of the polar cap of the sphere with radius r and cap height h
    """
    return pi * (3 * r - h) * h ** 2 / 3


def vol(r):
    """
    Volume of sphere with radius r
    """
    return 4 * pi * r ** 3 / 3


def s2_minus_s1(r1, r2, d):

    """
    Volume of S2 outside of S1
    r1: radius of S1
    r2: radius of S2
    d: distance between center of S1 and center of S2
    """

    # S1 and S2 not in contact
    if d > r1 + r2:
        return vol(r2)

    # S1 sits inside S2
    elif (r2 > d + r1):
        return vol(r2) - vol(r1)

    # S2 sits inside S1
    elif (r1 > d + r2):
        return 0.

    # Center of S2 is outside S1
    elif d > r1:
        h1 = (r2 ** 2 - (d - r1) ** 2) / (2 * d)
        h2 = r1 - h1 + r2 - d
        return vol(r2) - cap(r1, h1) - cap(r2, h2)

    # Center of S2 is inside S1
    elif d <= r1:
        h1 = (r2 ** 2 - (r1 - d) ** 2) / (2 * d)
        h2 = d - r1 + h1 + r2
        return cap(r2, h2) - cap(r1, h1)

In [11]:
# Other functions

def get_structure(file):
    """
    Use Bio.PDB to parse protein structure.
    """

    id = file.split("AF-")[1].split("-model_v1")[0]

    if file.endswith('.gz'):
        with gzip.open(file, 'rt') as handle:
            return PDBParser().get_structure(id=id, file=handle)[0]
    else:
        with open(file, 'r') as handle:
            return PDBParser().get_structure(id=id, file=handle)[0]


def get_dist_matrix(chain) :
    """
    Compute the distance matrix between C-alpha of a protein.
    """

    m = np.zeros((len(chain), len(chain)), float)

    for i, res1 in enumerate(chain) :
        for j, res2 in enumerate(chain) :
            m[i, j] = abs(res1["CA"] - res2["CA"])

    return m


def get_contact_map(chain, distance=10):
    """
    Compute the contact map between C-alpha of a protein.
    """

    dist_matrix = get_dist_matrix(chain)

    return dist_matrix < distance


def get_prob_contact(pae_value, dmap_value, distance=10):
    """
    Get probability of contact considering the distance
    between residues in the predicted structure and the
    Predicted Aligned Error (PAE).
    """

    if pae_value == 0:

        if dmap_value < distance:
            return 1
        else:
            return 0

    else:
        # Get the volume of res2 outside of res1
        vol_s2_out_s1 = s2_minus_s1(r1=distance, r2=pae_value, d=dmap_value)

        # Get the probability that s2 is out of s1
        p_s2_in_s1 = vol_s2_out_s1 / vol(pae_value)

        return 1 - p_s2_in_s1


def get_prob_cmap(chain, pae, distance=10) :
    """
    Compute the probabilities that each pair of residue in a protein are
    in contact taking into account the Predicted Aligned Error (PAE) and
    the PDB structure predicted by AlphaFold 2
    """

    m = np.zeros((len(chain), len(chain)), float)

    for i, res1 in enumerate(chain):
        for j, res2 in enumerate(chain):
            d = abs(res1["CA"] - res2["CA"])
            m[i, j] = get_prob_contact(pae[i, j], d, distance)

    return m

In [24]:
# New method and comparison

def get_prob_cmap_alt(chain, pae, distance=10) :
    """
    Compute the probabilities that each pair of residue in a protein are
    in contact taking into account the Predicted Aligned Error (PAE) and
    the PDB structure predicted by AlphaFold 2
    """

    m = np.zeros((len(chain), len(chain)), float)

    for i, res1 in enumerate(tqdm(chain, desc="Residue i")):
        for j, res2 in enumerate(chain):
            d = abs(res1["CA"] - res2["CA"])
            m[i, j] = 1 if i == j else contact_probability(pae[i, j], distance, d)

    return m


def compare_pcmaps(gene, seq_df, dataset_dir):
    """
    Compute cmaps in two alternative approaches and Person 
    correlation between the resulting matrices.
    """
    
    uni_id = seq_df[seq_df["Gene"] == gene].Uniprot_ID.values[0]
    pdb_path = f"{pdb_dir}/AF-{uni_id}-F1-model_v4.pdb"
    pae_path = f"{pae_dir}/{uni_id}-F1-predicted_aligned_error.npy"
    
    pdb = get_structure(pdb_path)["A"]
    pae = np.load(pae_path)
    pcmap = get_prob_cmap(pdb, pae)
    
    pcmap_alt = get_prob_cmap_alt(pdb, pae)

    corr, pval = pearsonr(pcmap.flatten(), pcmap_alt.flatten())
    print(f"Pearson correlation: {corr:.4f}, p-value: {pval:.4g}")
    
    return pcmap, pcmap_alt, corr, pval

In [22]:
dataset_dir = "/data/bbg/nobackup/scratch/oncodrive3d/datasets_mane_240506"
pdb_dir = f"{dataset_dir}/pdb_structures"
cmaps_dir = f"{dataset_dir}/prob_cmaps"
pae_dir = f"{dataset_dir}/pae"

seq_df = pd.read_table(f"{dataset_dir}/seq_for_mut_prob.tsv")

In [25]:
pcmap, pcmap_alt, corr, pval = compare_pcmaps("TP53", seq_df, dataset_dir)

Residue i:   3%|▎         | 13/393 [01:43<50:32,  7.98s/it]

In [ ]:
pcmap2, pcmap_alt2, corr2, pval2 = compare_pcmaps("KRAS", seq_df, dataset_dir)

In [ ]:
pcmap3, pcmap_alt3, corr3, pval3 = compare_pcmaps("CTNNB1", seq_df, dataset_dir)

In [ ]:
pcmap3, pcmap_alt3, corr3, pval3 = compare_pcmaps("PIK3CA", seq_df, dataset_dir)